### Bibliotecas

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box
import osmnx as ox
import networkx as nx
from itertools import combinations
import multiprocessing as mp

print(f'Utilizando a versão {ox.__version__} do OSMNX')

print(f'Eu tenho {mp.cpu_count()} CPUs disponíveis')

Utilizando a versão 1.9.3 do OSMNX
Eu tenho 8 CPUs disponíveis


### Parâmetros

In [2]:
caminho = '/Users/marcelofernandes/Library/CloudStorage/GoogleDrive-marcelo.fernandes@alumni.usp.br/.shortcut-targets-by-id/1M--OnzbTYagrNv5Ss9fjWlBxCMmasz-Y/10_Mestrado_2021_Marcelo Fernandes/6_Qualificação/Mapas temáticos e figuras/BACIA_HIDROGRAFICA/SIRGAS_BACIAHIDROGRAFICA.shp'

my_crs = '4326'  # Verificar em EPSG.io um projeção que te dê o resultado em metros

pesquisa_OD = '/Users/marcelofernandes/Library/CloudStorage/GoogleDrive-marcelo.fernandes@alumni.usp.br/.shortcut-targets-by-id/1M--OnzbTYagrNv5Ss9fjWlBxCMmasz-Y/10_Mestrado_2021_Marcelo Fernandes/8_Dados/Pesquisa OD 2017/OD-2017/Mapas-OD2017/Shape-OD2017/Zonas_2017_region.shp'
plt.style.use('ggplot')

arquivo_matrizOD = '/Users/marcelofernandes/Library/CloudStorage/GoogleDrive-marcelo.fernandes@alumni.usp.br/.shortcut-targets-by-id/1M--OnzbTYagrNv5Ss9fjWlBxCMmasz-Y/10_Mestrado_2021_Marcelo Fernandes/4_Códigos/Dados/OD-2017/Tabelas-OD2017/Tab25_OD2017.xlsx'

verbose = False  # Flag para pular algumas atividades, com o objetivo de economizar memória.

ox.settings.log_console = True
ox.settings.use_cache = True

In [6]:
# Carregar o grafo de um arquivo GraphML e o bounding box
try:
    G_baseline = ox.load_graphml(filepath='network_baseline_vel_updated.graphml')
    print('Grafo carregado com sucesso!')
except:
    print('Arquivo não encontrado')

try:
    bounding_box_baseline = gpd.read_file('bounding_box_bacia.json')
    print('Bounding box carregado com sucesso!')
except:
    print('Bounding box não encontrado')
    
node_baseline, edges_baseline = ox.graph_to_gdfs(G_baseline)

Grafo carregado com sucesso!
Bounding box carregado com sucesso!


### Carregar os pontos da pesquisa OD referentes a bacia

In [5]:
# Read the shapefile
gdf_OD = gpd.read_file(pesquisa_OD)

# Verifica o sistema de coordenadas
gdf_OD.crs

# Atribuição do esquema de projeção
gdf_OD = gdf_OD.to_crs(my_crs)

# Filtrando para o munícipio de São Paulo
gdf_indices = gdf_OD.index[gdf_OD['NomeMunici'] == 'São Paulo']
gdf_OD_SP = gdf_OD.loc[gdf_indices]

# Obtendo os centroides
gdf_OD_SP["centroid"] = gdf_OD_SP["geometry"].centroid

if verbose:
    gdf_OD_SP['centroid'].explore(
        #                            column = "NumeroZona",
        #                            tooltip = "NumeroZona",
        #                            popup = True,
        #                            tiles = "CartoDB dark_matter",
        #                            cmap = "inferno_r")
    )

/var/folders/8y/8z4lzw294m5c0kf05d8ww4r00000gn/T/ipykernel_27691/3141057998.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_OD_SP["centroid"] = gdf_OD_SP["geometry"].centroid


In [7]:
centroides_df = gpd.GeoDataFrame(gdf_OD_SP[['NumeroZona', 'NomeZona']], geometry=gdf_OD_SP['centroid'])
centroides_dentro = gpd.sjoin(centroides_df, bounding_box_baseline, how='inner', predicate='within')

# Remover a coluna de índice criada pelo sjoin
centroides_dentro = centroides_dentro.drop(columns=['index_right'])

# Redefinir o índice
centroides_dentro = centroides_dentro.reset_index(drop=True)

centroides_dentro.shape

(65, 3)

In [35]:
# 1. Aproximar os pontos de centroides_dentro à rede
centroides_dentro['nearest_node_baseline'] = centroides_dentro['geometry'].apply(
    lambda point: ox.distance.nearest_nodes(G_baseline, point.x, point.y)
)

### Calcular o baseline (caminhos mínimos, sem chuva)

In [18]:
def listar_atributos_arestas(G):
    atributos = set()
    for _, _, dados in G.edges(data=True):
        atributos.update(dados.keys())
    return atributos

In [32]:
# Função para adicionar tempo de viagem às arestas
def add_edge_travel_times(G, referencia):
    for u, v, key, data in G.edges(keys=True, data=True):
        length = data.get('length', 0)  # Certifique-se de que length esteja definido
        vel_ref = data.get(referencia, 0)  # Certifique-se de que vel_ref esteja definido
        
        if vel_ref == 0:
            travel_time_ref = float('inf')  # Indica que o tempo de viagem é infinito
        else:
            travel_time_ref = (length*3.6) / vel_ref  #*(3600/1000)  # Convertendo para segundos
        nome = 'travel_time_' + referencia
        G.edges[u, v, key][nome] = travel_time_ref
        
        edges_to_remove = [(u, v) for u, v, data in G.edges(data=True) if data.get(nome) == float('inf')]

        # Remover as arestas com tempo de viagem igual a infinito
        G.remove_edges_from(edges_to_remove)
    return G

In [33]:
# Adicionar velocidade aos eixos e calcular o tempo de viagem
referencia = 'speed_kph'
G_mod = add_edge_travel_times(G_baseline, referencia)

In [39]:
import pandas as pd
import itertools
import time

# Carregar a planilha no DataFrame
# Considerando que a primeira linha contém os cabeçalhos e as zonas começam de 1
matriz_od_25 = pd.read_excel(arquivo_matrizOD, header=7, index_col=0)
# Exibir as primeiras linhas do DataFrame para verificação
print(matriz_od_25.head())

              1           2    3           4           5           6  \
NaN         NaN         NaN  NaN         NaN         NaN         NaN   
1    128.247095   86.383235  0.0  191.877452  105.088345  151.710615   
2     86.383235    0.000000  0.0    0.000000  612.400416  256.941702   
3      0.000000    0.000000  0.0    0.000000  194.857696    0.000000   
4      0.000000  214.428586  0.0  372.251396  455.222595    0.000000   

              7           8          9   10  ...  509  510  511  512  513  \
NaN         NaN         NaN        NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN   
1    234.477033    0.000000   0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0   
2      0.000000    0.000000  92.275009  0.0  ...  0.0  0.0  0.0  0.0  0.0   
3      0.000000    0.000000   0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0   
4      0.000000  159.375738   0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0   

     514  515        516  517         Total  
NaN  NaN  NaN        NaN  NaN           NaN  
1    0.0  0.

In [34]:
# 1. Função para obter o número de viagens da matriz OD
def get_number_of_trips(node1, node2, referencia, od_matrix):
    zone1 = centroides_dentro.loc[centroides_dentro[referencia] == node1, 'NumeroZona'].values[0]
    zone2 = centroides_dentro.loc[centroides_dentro[referencia] == node2, 'NumeroZona'].values[0]
    return od_matrix.loc[zone1, zone2]

In [52]:
# Definição da função para encontrar o caminho mínimo
def caminho_minimo(G, source, target, weight):
    # Encontrar o caminho mais curto
    path = nx.shortest_path(G, source=source, target=target, weight=weight)
    path_cost = nx.path_weight(G, path, weight=weight)
    return path, path_cost

In [54]:
 def calcula_caminho_minimo(gdf_OD, referencia_OD, G, referencia):
    data = []
    data_batch = []
    start_time = time.time()
    for node1, node2 in itertools.product(gdf_OD[referencia_OD], repeat=2):
        if node1 == node2:
            continue  # Ignorar pares onde origem e destino são iguais
        try:
            # Obter o número de viagens entre node1 e node2
            num_trips = get_number_of_trips(node1, node2, 'nearest_node_baseline', matriz_od_25)
            if num_trips > 0:
                # Calcular o caminho mínimo pelo tempo de viagem ponderado
                if node1 in G.nodes and node2 in G.nodes:
                    # Calcular o caminho mais curto
                    caminho_com, tempo_com = caminho_minimo(G, node1, node2, referencia)
                    weighted_time_rain = tempo_com * num_trips
                else:
                    print(f"Nó(s) {node1} ou {node2} não encontrado(s) no grafo.")
                    continue
                data_batch.append({
                    'index': len(data) + 1,
                    'Origem': node1,
                    'Destino': node2,
                    'Num_viagens': num_trips,
                    'Tempo com chuva (min)': tempo_com / 60,
                    'Nós com chuva': len(caminho_com),
                    'Tempo ponderado com chuva (min)': weighted_time_rain / 60
                        })

        except nx.NetworkXNoPath:
            # Se não houver caminho entre os nós, ignore esse par
            continue
            
    # Parar o cronômetro e calcular o tempo de processamento
    end_time = time.time()
    processing_time = end_time - start_time
    print(f"Tempo de processamento: {processing_time / 60:.2f} minutos")
    print(f'Foram calculadas {len(index)} viagens')
    print(f'O tempo médio de viagens entre os pares OD foi de {}')
            
    # Criar DataFrame e salvar em CSV
    tempos_e_viagens_arquivo = pd.DataFrame(data_batch)
    #output_file = output_dir / f'tempo_e_viagens_{first_unprocessed_file}.csv'
    #tempos_e_viagens_arquivo.to_csv(output_file, index=False)
    #print(f"Dados salvos em {output_file}")
    #tempos_e_viagens_arquivo
    return tempos_e_viagens_arquivo

In [55]:
tempos_e_viagens_arquivo = calcula_caminho_minimo(centroides_dentro,'nearest_node_baseline', G_mod, 'travel_time_speed_kph')

Tempo de processamento: 0.15 minutos


In [56]:
tempos_e_viafens_arquivo

,index,Origem,Destino,Num_viagens,Tempo com chuva (min),Nós com chuva,Tempo ponderado com chuva (min)
0,1,340185206,4499663530,1322.714416,3.340624,15,4418.691390
1,1,340185206,458457689,696.854080,3.045976,12,2122.600524
2,1,340185206,129958225,520.898998,7.474322,35,3893.366892
3,1,340185206,130009969,433.082541,6.091142,31,2637.967428
4,1,340185206,433999406,126.223527,9.661934,43,1219.563433
...,...,...,...,...,...,...,...
846,1,1645957962,1071641620,522.664724,8.881170,42,4641.874270
847,1,1645957962,1040335345,459.687636,4.869896,29,2238.631162
848,1,1645957962,6358057222,1482.042049,3.078090,18,4561.859254
849,1,1645957962,1095092347,343.050791,6.627123,24,2273.439785
